# Environmental LLM - Complete Training & RAG Chat
## GPT-2 + Live OpenWeather API + SerpAPI Internet Search

In [1]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q transformers accelerate
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import GPT2LMHeadModel, GPT2Tokenizer, get_cosine_schedule_with_warmup
import json, random, os, requests
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

E0000 00:00:1765509599.443590     171 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765509599.450285     171 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Device: cuda


In [3]:
# CONFIG
DATA_PATH = '/kaggle/input/nothingg/real_api_training_data.jsonl'
CKPT_DIR = '/kaggle/working/checkpoints'
MAX_LEN, BATCH, GRAD_ACC = 256, 8, 4
STEPS, LR, WARMUP = 6000, 5e-5, 300
os.makedirs(CKPT_DIR, exist_ok=True)

In [4]:
# DATASET
class EnvDataset(Dataset):
    def __init__(self, path, tokenizer, train=True):
        self.tokenizer = tokenizer
        samples = [json.loads(l) for l in open(path)]
        idx = int(len(samples)*0.9)
        self.samples = samples[:idx] if train else samples[idx:]
        print(f"{'Train' if train else 'Val'}: {len(self.samples):,}")
    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        s = self.samples[i]
        text = f"User: {s['input']}\nAssistant: {s['output']}"
        enc = self.tokenizer(text, max_length=MAX_LEN, truncation=True, padding='max_length', return_tensors='pt')
        return enc.input_ids.squeeze()

In [5]:
# LOAD MODEL & DATA
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
print(f'Model: {sum(p.numel() for p in model.parameters())/1e6:.0f}M params')

train_ds = EnvDataset(DATA_PATH, tokenizer, train=True)
val_ds = EnvDataset(DATA_PATH, tokenizer, train=False)
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH)
print('Data loaded!')

Model: 124M params
Train: 18,360
Val: 2,040
Data loaded!


In [6]:
# TRAINING
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=0.01)
scheduler = get_cosine_schedule_with_warmup(optimizer, WARMUP, STEPS)
model.train()
step, best_val = 0, float('inf')
pbar = tqdm(total=STEPS)

for epoch in range(20):
    for batch in train_loader:
        ids = batch.to(device)
        loss = model(ids, labels=ids).loss / GRAD_ACC
        loss.backward()
        if (step+1) % GRAD_ACC == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        step += 1
        pbar.update(1)
        pbar.set_postfix(loss=f'{loss.item()*GRAD_ACC:.4f}')
        if step % 500 == 0:
            model.eval()
            val_loss = sum(model(b.to(device), labels=b.to(device)).loss.item() for b in val_loader)/len(val_loader)
            print(f'\nStep {step}: val_loss={val_loss:.4f}')
            if val_loss < best_val:
                best_val = val_loss
                torch.save(model.state_dict(), f'{CKPT_DIR}/best_model.pt')
                print('Saved best!')
            model.train()
        if step % 1500 == 0:
            torch.save(model.state_dict(), f'{CKPT_DIR}/ckpt_{step}.pt')
        if step >= STEPS: break
    if step >= STEPS: break
pbar.close()
print(f'Done! Best val_loss: {best_val:.4f}')

  8%|▊         | 500/6000 [04:53<54:36,  1.68it/s, loss=0.3317]  


Step 500: val_loss=0.2303


  8%|▊         | 501/6000 [05:46<25:04:49, 16.42s/it, loss=0.3317]

Saved best!


 17%|█▋        | 1000/6000 [10:45<49:57,  1.67it/s, loss=0.1539]  

...

Saved best!


100%|██████████| 6000/6000 [1:09:26<00:00,  1.66it/s, loss=0.0423]  


Step 6000: val_loss=0.0346
Saved best!


100%|██████████| 6000/6000 [1:10:20<00:00,  1.42it/s, loss=0.0423]

Done! Best val_loss: 0.0346


In [7]:
# === RAG SYSTEM: LIVE DATA + INTERNET SEARCH ===
OPENWEATHER_API_KEY = '84fa0ee0ff4c054334d5ee962dd7c870'
SERPAPI_KEY = '3435cc6097e3d0fe22e5846418cafd25ba4ab916526df5855cce55a9dbc6df64'

CITIES = {
    'kolkata': (22.57, 88.36), 'delhi': (28.61, 77.21), 'mumbai': (19.08, 72.88),
    'bangalore': (12.97, 77.59), 'chennai': (13.08, 80.27), 'hyderabad': (17.38, 78.49),
    'siliguri': (26.73, 88.40), 'darjeeling': (27.04, 88.27), 'durgapur': (23.52, 87.31),
    'asansol': (23.67, 86.95), 'howrah': (22.59, 88.26), 'malda': (25.01, 88.14),
}

def get_live_aqi(city):
    '''Fetch LIVE AQI from OpenWeather'''
    if city.lower() not in CITIES: return None
    lat, lon = CITIES[city.lower()]
    try:
        url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}'
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            data = r.json()['list'][0]['components']
            pm25 = data.get('pm2_5', 0)
            # India AQI calculation
            bp = [(0,30,0,50),(31,60,51,100),(61,90,101,200),(91,120,201,300),(121,250,301,400),(251,500,401,500)]
            aqi = 500
            for c_lo, c_hi, i_lo, i_hi in bp:
                if c_lo <= pm25 <= c_hi:
                    aqi = int(((i_hi-i_lo)/(c_hi-c_lo))*(pm25-c_lo)+i_lo)
                    break
            cat = 'Good' if aqi<=50 else 'Satisfactory' if aqi<=100 else 'Moderate' if aqi<=200 else 'Poor' if aqi<=300 else 'Very Poor' if aqi<=400 else 'Severe'
            return {'aqi': aqi, 'category': cat, 'pm25': pm25, 'pm10': data.get('pm10', 0)}
    except: pass
    return None

def get_live_weather(city):
    '''Fetch LIVE weather from OpenWeather'''
    if city.lower() not in CITIES: return None
    lat, lon = CITIES[city.lower()]
    try:
        url = f'http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric'
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            data = r.json()
            return {'temp': data['main']['temp'], 'humidity': data['main']['humidity'], 'desc': data['weather'][0]['description']}
    except: pass
    return None

def search_internet(query):
    '''Search the internet using SerpAPI'''
    try:
        url = f'https://serpapi.com/search.json?q={query}&api_key={SERPAPI_KEY}'
        r = requests.get(url, timeout=15)
        if r.status_code == 200:
            data = r.json()
            results = []
            for item in data.get('organic_results', [])[:3]:
                results.append({'title': item.get('title',''), 'snippet': item.get('snippet','')})
            return results
    except: pass
    return None

print('RAG System Ready!')

RAG System Ready!


In [8]:
# === RAG-ENHANCED GENERATION ===
@torch.no_grad()
def rag_generate(prompt, max_tokens=150, temp=0.8):
    model.eval()
    context = ''
    
    # 1. Extract city and get LIVE data
    for city in CITIES:
        if city in prompt.lower():
            aqi = get_live_aqi(city)
            weather = get_live_weather(city)
            if aqi:
                context += f'[LIVE DATA: {city.title()} AQI={aqi["aqi"]} ({aqi["category"]}), PM2.5={aqi["pm25"]:.1f}]\n'
            if weather:
                context += f'[WEATHER: {weather["temp"]}°C, {weather["humidity"]}% humidity, {weather["desc"]}]\n'
            break
    
    # 2. Search internet if needed
    search_keywords = ['news', 'latest', 'research', 'study', 'why', 'how', 'cause', 'effect', 'climate']
    if any(kw in prompt.lower() for kw in search_keywords):
        results = search_internet(prompt + ' India environment')
        if results:
            context += f'[WEB SEARCH: {results[0]["snippet"][:150]}]\n'
    
    # 3. Generate with context
    full_prompt = f'{context}User: {prompt}\nAssistant:' if context else f'User: {prompt}\nAssistant:'
    ids = tokenizer.encode(full_prompt, return_tensors='pt').to(device)
    if ids.size(1) > 400: ids = ids[:, -400:]
    
    for _ in range(max_tokens):
        out = model(ids)
        logits = out.logits[:, -1, :] / temp
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, 1)
        ids = torch.cat([ids, next_id], dim=1)
        if next_id.item() == tokenizer.eos_token_id: break
    
    return tokenizer.decode(ids[0], skip_special_tokens=True).split('Assistant:')[-1].strip()

# Load best model
model.load_state_dict(torch.load(f'{CKPT_DIR}/best_model.pt'))
print('Model loaded!')

Model loaded!


In [9]:
# === TEST RAG SYSTEM ===
print('\n' + '='*60)
print('Testing RAG System with Live Data + Internet Search')
print('='*60 + '\n')

test_queries = [
    'What is the current AQI in Kolkata?',
    'Compare air quality in Delhi vs Darjeeling',
    'Why is pollution increasing in India?',
    'Latest news about air quality'
]

for q in test_queries:
    print(f'Q: {q}')
    response = rag_generate(q)
    
    # Add live data display
    for city in CITIES:
        if city in q.lower():
            aqi = get_live_aqi(city)
            if aqi:
                response += f'\n\n📊 LIVE DATA ({city.title()}):'
                response += f'\n🔴 AQI: {aqi["aqi"]} ({aqi["category"]})'
                response += f'\n💨 PM2.5: {aqi["pm25"]:.1f} μg/m³'
            break
    
    print(f'A: {response}')
    print('-'*60)


Testing RAG System with Live Data + Internet Search

Q: What is the current AQI in Kolkata?
A: **Kolkata, West Bengal - Current Air Quality**

📊 **AQI:** 457 (Severe)
🔴 **PM2.5:** 398.1 μg/m³
🟠 **PM10:** 446.9 μg/m³

🌡️ **Weather:**
- Temperature: 15.2°C
- Humidity: 59%
- Wind: 2.1 m/s

⚠️ **Health Impact:** Serious health effects. Avoid outdoor activities.

💡 **Advisory:** Stay indoors. Keep windows closed. Use air purifiers. Seek medical attention if breathing issues occur.

📊 LIVE DATA (Kolkata):
🔴 AQI: 424 (Severe)
💨 PM2.5: 309.9 μg/m³
------------------------------------------------------------
Q: Compare air quality in Delhi vs Darjeeling
A: **Air Quality Comparison**

| City | AQI | PM2.5 | Category |
|------|-----|-------|----------|
| Delhi | 252 | 106 | Poor |
| Darjeeling | 460 | 398 | Severe |

🏆 **Winner:** Delhi has better air quality
📊 **Difference:** 150 AQI points

Delhi is significantly cleaner than the other city.

📊 LIVE DATA (Delhi):
🔴 AQI: 299 (Poor)
💨 PM2.5: 120

In [ ]:
# === INTERACTIVE RAG CHAT ===
print('\n' + '='*60)
print('🌍 Environmental RAG Chat')
print('Features: Live AQI | Live Weather | Internet Search')
print('Commands: "search: <query>" for web search')
print('Type "quit" to exit')
print('='*60 + '\n')

while True:
    q = input('You: ').strip()
    if q.lower() in ['quit','exit','q']: 
        print('Goodbye! 👋')
        break
    if not q: continue
    
    # Direct web search
    if q.lower().startswith('search:'):
        query = q[7:].strip()
        print(f'\n🔍 Searching: {query}')
        results = search_internet(query)
        if results:
            for i, r in enumerate(results, 1):
                print(f'{i}. {r["title"]}')
                print(f'   {r["snippet"][:200]}...')
        else:
            print('No results found.')
        print()
        continue
    
    # RAG-enhanced response
    response = rag_generate(q)
    
    # Add live data
    for city in CITIES:
        if city in q.lower():
            aqi = get_live_aqi(city)
            weather = get_live_weather(city)
            if aqi:
                response += f'\n\n📊 LIVE ({city.title()}): AQI={aqi["aqi"]} ({aqi["category"]}), PM2.5={aqi["pm25"]:.1f}'
            if weather:
                response += f'\n🌡️ Weather: {weather["temp"]}°C, {weather["humidity"]}% humidity'
            break
    
    print(f'\n🤖 Bot: {response}\n')


🌍 Environmental RAG Chat
Features: Live AQI | Live Weather | Internet Search
Commands: "search: <query>" for web search
Type "quit" to exit



You:  <WHAT IS THE FUTURE FORECAST OF KOLKATA>



🤖 Bot: **Kolkata, West Bengal - Current Air Quality**

📊 **AQI:** 424 (Severe)
🔴 **PM2.5:** 317.7 μg/m³
🟠 **PM10:** 343.2 μg/m³

🌡️ **Weather:**
- Temperature: 15.01°C
- Humidity: 63%
- Wind: 1.01 m/s

⚠️ **Health Impact:** Serious health effects. Avoid outdoor activities.

💡 **Advisory:** Stay indoors. Keep windows closed. Use air purifiers. Seek medical attention if breathing issues occur.

📊 LIVE (Kolkata): AQI=424 (Severe), PM2.5=309.9
🌡️ Weather: 19.95°C, 73% humidity



You:  <Forecast of Kolkata temperature>



🤖 Bot: **Kolkata, West Bengal - 5-Day Forecast**

**Day 1 (Fri 12 Dec):**
- 🌡️ Temp: 17°C | 💧 Humidity: 78%
- ☁️ Condition: Clear Sky
- 🔴 AQI: 426 (Severe)

**Day 2 (Fri 12 Dec):**
- 🌡️ Temp: 22°C | 💧 Humidity: 52%
- ☁️ Condition: Clear Sky
- 🔴 AQI: 426 (Severe)

**Day 3 (Fri 12 Dec):**
- 🌡️ Temp: 27°C | 💧 Humidity: 24

📊 LIVE (Kolkata): AQI=424 (Severe), PM2.5=309.9
🌡️ Weather: 19.95°C, 73% humidity

